## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
diabetes = datasets.load_diabetes()

x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size = 0.3, random_state = 87)

clf = GradientBoostingRegressor(random_state = 87)

In [6]:
# 使用預設參數訓練得到的結果
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("MSE: ", metrics.mean_squared_error(y_test, y_pred))

MSE:  3144.739284050225


In [13]:
# 設定要訓練的超參數組合
n_estimators = [10, 100, 1000]
max_depth = [1, 4, 9]
min_samples_leaf = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf)

# 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs = -1, verbose=1)
# verbose：日誌顯示，0為不在標準輸出流輸出日誌信息，1為輸出進度條記錄，2為每個epoch輸出一行記錄

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)
# 預設會跑 5-fold cross-validadtion，總共 27 種參數組合，總共要 train 135 次模型

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed:   21.7s finished


In [14]:
grid_result.best_params_

{'max_depth': 1, 'min_samples_leaf': 5, 'n_estimators': 100}

In [15]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                         n_estimators=grid_result.best_params_['n_estimators'], 
                                         min_samples_leaf = grid_result.best_params_['min_samples_leaf'])
# 重新訓練
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [16]:
print("Improved MSE: ", metrics.mean_squared_error(y_test, y_pred))

Improved MSE:  2936.776540847153


MSE: 3144 -> 2936